In [ ]:
#Import word2vec
from gensim.models import Word2Vec

# Define Neo4j connections
from neo4j import GraphDatabase
import pandas as pd

host = 'bolt://localhost:7687'
user = 'neo4j'
password = 'letmein'
driver = GraphDatabase.driver(host,auth=(user, password))

## Graph import

Today we will be using the Spoonacular Food Dataset that is available on Kaggle. It contains nutritional information alongside the ingredients used in 1600+ dishes. Unfortunately, it contains no recipe for sourdough bread.

There are three types of nodes in our graph schema. A dish consists of one or more ingredients, which we represent as a connection between a dish and its ingredients. Recipes fall into categories or types such as lunch, breakfast, and so on.
We use the <code>apoc.schema.assert</code> procedure to define the graph schema. It allows us to describe multiple indexes and unique constraints in a single query.

Before we can execute the import query, we have to download the dataset and copy it to the Neo4j import folder. In the import query, we do a tiny bit of preprocessing as we lowercase the names of ingredients and replace dash characters (-) with whitespace.

In [ ]:
graph_schema_query = """

CALL apoc.schema.assert( 
    // define indexes 
    null, 
    // define unique constraints 
    {Ingredient:['name'], Dish:['id'], DishType:['name']})

"""

graph_import_query = """

LOAD CSV WITH HEADERS FROM "file:///newfood.csv" as row 
CREATE (d:Dish{id:row.id}) 
SET d += apoc.map.clean(row, ['id','dishTypes','ingredients'],[]) 
FOREACH (i in split(row.ingredients,',') | MERGE (in:Ingredient{name:toLower(replace(i,'-',' '))}) 
                                           MERGE (in)<-[:HAS_INGREDIENT]-(d)) 
FOREACH (dt in split(row.dishTypes,',')  | MERGE (dts:DishType{name:dt}) 
                                           MERGE (dts)<-[:DISH_TYPE]-(d))

"""

with driver.session() as session:
    session.run(graph_schema_query)
    session.run(graph_import_query)

### Basic graph exploration

To start off, we will do a bit of graph exploration. Let's look at which ingredients are used the most.

In [ ]:
with driver.session() as session:
    results = session.run("""
    MATCH (n:Ingredient)
    RETURN n.name as ingredient, size((n)<--()) as mentions 
    ORDER BY mentions DESC
    LIMIT 10
    """)
pd.DataFrame([dict(result) for result in results])

Olive oil is by far the most popular as it is used in more than half of the recipes. Slowly following are garlic, salt, and butter. I didn't know that butter was so popular. I also found it quite surprising that anchovies are so widely used. Or maybe the dataset is just biased towards dishes that contain anchovies.

We could build an application on top of this dataset to search for recipes based on the desired ingredients we want to cook. I borrowed this cypher query from the What's cooking series written by Mark Needham and Lju Lazarevic. Let's say, for example, you want to eat zucchini and feta cheese today, but don't have any idea which recipe to go for. Luckily, our application could help us solve this problem with the following cypher query.

In [ ]:
with driver.session() as session:
    results = session.run("""
    WITH ["feta cheese", "zucchini"] as ingredients 
    MATCH (d:Dish) 
    WHERE all(i in ingredients WHERE exists( 
        (d)-[:HAS_INGREDIENT]->(:Ingredient {name: i}))) 
    RETURN d.title AS dish, 
           [(d)-[:HAS_INGREDIENT]->(i) | i.name] AS ingredients 
    ORDER BY size(ingredients)
    LIMIT 10
    """)
pd.DataFrame([dict(result) for result in results])

We could go with a salad or a fish. I think I'll pass and skip over to node2vec for dessert.

## Node2vec algorithm

The node2vec algorithm is relatively new. It was proposed only in the year 2016 by Jure Leskovac and Aditya Grover in an article node2vec: Scalable Feature Learning for Networks. To understand how it works, we must first understand the word2vec algorithm. Word2vec algorithm was proposed in the year 2013 by a team of researchers led by Tomas Mikolov at Google. It is a popular technique using neural networks to learn the word embedding. It takes a list of sentences as input and produces a vector or an embedding for each word that appears in the text corpus. Words with similar meanings should be closer in the embedding space. For example, apple and pear should be more similar than apple and car. There are two training algorithms for word2vec. The first method is called the continuous bag of words (CBOW), which uses the context of the word to predict a target term. The context is defined as words that appear near the target word in the text. The second method is called skip-gram. Instead of trying to predict the target word from the context, it tries to predict the context of a given term. If you want to learn more about the word2vec algorithm, there is plenty of good literature about it on the internet.


You might ask how we get from word2vec to node2vec. It is actually straightforward. Instead of using a list of sentences as input, we use a list of random walks. This is the only difference.

## Neo4j Graph Data Science library

Neo4j Graph Data Science library supports the random walk algorithm, which makes it very easy for us to implement the node2vec algorithm. If you need a quick refresher on how the GDS library works, you can check out my previous blog post. We will start by projecting the in-memory graph. We describe all three node labels and project relationships as undirected.

In [ ]:
with driver.session() as session:
    session.run("""CALL gds.graph.create('all', 
    '*', 
    {ALL_UNDIRECTED: {type:'*', orientation:'UNDIRECTED'}})""")

Now we are ready to train our first node2vec model. The process will consist of three parts:

Execute the random walk algorithm starting from each node in the graph
Feed the random walks to word2vec algorithm
Inspect results by looking at the most similar neighbors

The random walk algorithm has an optional start parameter, which can be used to define the starting node of the walk. We can also specify how long the walk should be with the steps setting and how many times it should be repeated with the walks parameter. Note that every time random walk is executed, we expect a different result.

We will use the Word2vec algorithm implementation in the gensim library. It also has a couple of hyperparameters we can define. Most notable are:

* size: Dimensionality of the embedding vectors
* window: Maximum distance between the current and predicted word
* min_count: The minimum count of words to consider when training the model; words with occurrence less than this count will be ignored.
* sg: The training algorithm: 1 for skip-gram; otherwise default CBOW

Check out the official documentation for more information about the word2vec hyperparameters

In [ ]:
# Define random walk query
random_walks_query = """

MATCH (node)
CALL gds.alpha.randomWalk.stream('all', {
  start: id(node),
  steps: 15,
  walks: 5
})
YIELD nodeIds
// Return the names or the titles
RETURN [id in nodeIds | 
    coalesce(gds.util.asNode(id).name, 
             gds.util.asNode(id).title)] as walks

"""
# Fetch data from Neo4j
with driver.session() as session:
    walks = session.run(random_walks_query)
# Train the word2vec model
clean_walks = [row['walks'] for row in walks]
model = Word2Vec(clean_walks, sg=1, window=5, size=100)
# Inspect results
model.most_similar('olive oil')

If I knew it was that easy, I would have written about the node2vec algorithm before. On the other hand, the results smell kind of fishy. I have no idea what is it with this dataset and anchovies. It seems like the recipes were mostly written by someone who really likes them. You will probably get quite different results though.

In the original node2vec paper, the authors defined two parameters that control the random walks execution. The first one is the return parameter.

>Return parameter, p. Parameter p controls the likelihood of immediately revisiting a node in the walk. Setting it to a high value (> max(q, 1)) ensures that we are less likely to sample an already visited node in the following two steps (unless the next node in the walk had no other neighbor). This strategy encourages moderate exploration and avoids 2-hop redundancy in sampling. On the other hand, if p is low (< min(q, 1)), it would lead the walk to backtrack a step (Figure 2) and this would keep the walk “local” close to the starting node u.

And the second parameter is called the in-out parameter.

>In-out parameter, q. Parameter q allows the search to differentiate between “inward” and “outward” nodes. Going back to Figure 2, if q > 1, the random walk is biased towards nodes close to node t. Such walks obtain a local view of the underlying graph with respect to the start node in the walk and approximate BFS behavior in the sense that our samples comprise of nodes within a small locality. In contrast, if q < 1, the walk is more inclined to visit nodes which are further away from the node t. Such behavior is reflective of DFS which encourages outward exploration. However, an essential difference here is that we achieve DFS-like exploration within the random walk framework. Hence, the sampled nodes are not at strictly increasing distances from a given source node u, but in turn, we benefit from tractable preprocessing and superior sampling efficiency of random walks. Note that by setting πv,x to be a function of the preceeding node in the walk t, the random walks are 2nd order Markovian.

In summary, the return parameter directs how often random walk backtracks a step or two. The in-out parameter controls if the random walk is more focused on local exploration, similar to BFS, or inclined more towards outward exploration like the DFS. Even though the random walk algorithm is still in the alpha tier, it supports these two node2vec parameters. Let's try them out in practice.

In [ ]:
# Define random walk query
random_walks_query = """

MATCH (node)
CALL gds.alpha.randomWalk.stream('all', {
  start: id(node),
  steps: 15,
  walks: 5,
  // define randomwalk node2vec params
  mode:'node2vec',
  inOut:0.6,
  return:1.0
})
YIELD nodeIds
// return name or title
RETURN [id in nodeIds | 
    coalesce(gds.util.asNode(id).name, 
             gds.util.asNode(id).title)] as walks

"""
# Fetch data from neo4j
with driver.session() as session:
    walks = session.run(random_walks_query)
# Train the word2vec model
clean_walks = [row['walks'] for row in walks]
model = Word2Vec(clean_walks, sg=1, window=5, size=100)
model.most_similar('olive oil')

Looking at results makes me hungry. It is really hard to say if the leg of lamb and olive oil should be regarded as similar ingredients. If we inspect the graph, out of eight recipes that include the leg of lamb, seven of those also use olive oil. By this logic, they are quite similar.

In our next example, we will show how to run the node2vec algorithm and store the result embeddings back to Neo4j. Instead of returning the titles of dishes and ingredients, we will return the internal Neo4j node ids. This will help us to link the results back to Neo4j efficiently.

In [ ]:
# Define random walk query
random_walks_query = """

MATCH (node)
CALL gds.alpha.randomWalk.stream('all', {
  start: id(node),
  steps: 15,
  walks: 5,
  mode:'node2vec',
  inOut:0.6,
  return:1.0
})
YIELD nodeIds
// return the string of internal ID of nodes
RETURN [id in nodeIds | toString(id)] as walks

"""
# fetch data from Neo4j
with driver.session() as session:
    walks = session.run(random_walks_query)
# Train model
clean_walks = [row['walks'] for row in walks]
model = Word2Vec(clean_walks, window=5, size=100, sg=1)

The embeddings are now available in the vocabulary of the word2vec model. We will store them to Neo4j in a single batch using the <code>UNWIND</code> cypher statement. If possible, try to avoid committing a single transaction per row as this is not very performant.

In [ ]:
store_embedding = """
UNWIND $data as row
MATCH (n)
WHERE id(n) = row.id
SET n.embedding = row.embedding
"""
embeddings = []
with driver.session() as session:
    for record in model.wv.vocab:
        id = record
        # Prepare data
        embeddings.append({'id':int(id), 'embedding': [float(x) for x in list(model.wv[id])]})
    # Store embeddings to Neo4j    
    session.run(store_embedding, {'data': embeddings})
        

Word2vec model uses the cosine similarity to find the most similar words. The Graph Data Science library also supports the Cosine similarity algorithm, which can be used to infer a similarity algorithm. As with all similarity algorithms, we have to fine-tune the <code>similarityCutoff</code> and <code>topK</code> parameters to get the best results. They directly influence how sparse the inferred similarity graph will be.

In [ ]:
cosine_similarity_algorithm = """

MATCH (node) 
WITH id(node) as id, node.embedding as weights 
WITH {item:id, weights: weights} as dishData 
WITH collect(dishData) as data 
CALL gds.alpha.similarity.cosine.write({
    nodeProjection: '*', 
    relationshipProjection: '*', 
    similarityCutoff:0.5, 
    topK:5, 
    data: data,
    writeRelationshipType:'COSINE_SIMILARITY'}) 
YIELD nodes, similarityPairs 
RETURN nodes, similarityPairs

"""

with driver.session() as session:
    results = session.run(cosine_similarity_algorithm)
pd.DataFrame([dict(result) for result in results])

To finish this analysis, we will inspect the community structure of the inferred network with the label propagation algorithm. As we are only interested in a rough outline of the community structure, we can use the stats mode of the algorithm to provide us some basic community structure statistics.

In [ ]:
with driver.session() as session:
    results = session.run("""CALL gds.labelPropagation.stats({
                            nodeProjection:'*',
                            relationshipProjection:'COSINE_SIMILARITY',
                            maxIterations:20})
                            YIELD communityCount, communityDistribution
                            RETURN communityCount,
                            apoc.math.round(communityDistribution.p50,2) as p50,
                            apoc.math.round(communityDistribution.p75,2) as p75,
                            apoc.math.round(communityDistribution.p90,2) as p90,
                            apoc.math.round(communityDistribution.p90,2) as p95,
                            apoc.math.round(communityDistribution.mean,2) as mean,
                            apoc.math.round(communityDistribution.max,2) as max""")
pd.DataFrame([dict(result) for result in results])

The label propagation algorithm found 123 groups in the similarity network. Most of them have less than 40 members. There are a handful of massive communities with the largest containing 625 members.

## Conclusion

The node2vec algorithm is a useful way of learning low-dimensional representations of the nodes in a graph that can be used downstream in a machine learning pipeline. During this blog post, I realized that changing the random walk algorithm parameters as well as the word2vec hyperparameters can produce very different results. Play around with them and see what works best for you.